In [ ]:
!pip install transformers sentencepiece datasets evaluate rouge_score
!pip install datasets

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.2 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=8ab56d839af89a046f6fa6e65a30060e403b059f74a367eb024f21cae46cf513
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
drive_model_dir = "/content/drive/MyDrive/t5-legal-summarizer"
drive_model_file = f"{drive_model_dir}/t5-legal-summarizer.model"

In [ ]:
import os, torch
from datasets import load_dataset
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)
import evaluate
import pandas as pd
import matplotlib.pyplot as plt
from transformers import LEDTokenizer, LEDForConditionalGeneration
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

In [ ]:
dataset = load_dataset("d0r1h/ILC")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


ValueError: Invalid pattern: '**' can only be an entire path component

In [ ]:
if os.path.exists(drive_model_dir):
    try:
        print("Loading model from Google Drive directory...")
        tokenizer = T5Tokenizer.from_pretrained(drive_model_dir)
        model = T5ForConditionalGeneration.from_pretrained(drive_model_dir)
    except:
        print("Failed to load full model. Trying to load from .model state file...")
        tokenizer = T5Tokenizer.from_pretrained("t5-base")
        model = T5ForConditionalGeneration.from_pretrained("t5-base")
        checkpoint = torch.load(drive_model_file)
        model.load_state_dict(checkpoint['model_state_dict'])
        print("Loaded from .model file")
else:
    print("Downloading and saving new model...")
    tokenizer = T5Tokenizer.from_pretrained("t5-base")
    model = T5ForConditionalGeneration.from_pretrained("t5-base")
    os.makedirs(drive_model_dir, exist_ok=True)
    tokenizer.save_pretrained(drive_model_dir)
    model.save_pretrained(drive_model_dir)
    torch.save({
        "model_state_dict": model.state_dict(),
        "config": model.config.to_dict(),
        "tokenizer": tokenizer.get_vocab()
    }, drive_model_file)


Loading model from Google Drive directory...


# ABOVE CODE TO RUN IF CRASH HAPPENS

In [ ]:
046a6a512357fd8175bda8c5c8fa03a952a495bf

In [ ]:
def preprocess_function(examples):
    inputs = ["summarize: " + text for text in examples["Case"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(examples["Summary"], max_length=150, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)


In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./t5-legal-summarizer",
    eval_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
    save_total_limit=2,
    predict_with_generate=True,
)


In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
)

<ipython-input-26-cc6dbdfa0844>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,5.711900,5.763082
2,5.471200,5.615032
3,5.278000,5.488533
4,5.107900,5.413004
5,4.957200,5.319380
6,4.850400,5.298864
7,4.742700,5.264151
8,4.684100,5.240685
9,4.642000,5.226212
10,4.588000,5.222058


TrainOutput(global_step=5150, training_loss=4.991932835254854, metrics={'train_runtime': 5403.6443, 'train_samples_per_second': 3.809, 'train_steps_per_second': 0.953, 'total_flos': 9549532543057920.0, 'train_loss': 4.991932835254854, 'epoch': 10.0})

In [ ]:
tokenizer.save_pretrained(drive_model_dir)
model.save_pretrained(drive_model_dir)
torch.save({
    "model_state_dict": model.state_dict(),
    "config": model.config.to_dict(),
    "tokenizer": tokenizer.get_vocab()
}, drive_model_file)

# BELOW

In [ ]:
def generate_summary(text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    inputs = tokenizer("summarize: " + text, return_tensors="pt", truncation=True, max_length=512).to(device)
    summary_ids = model.generate(
    inputs["input_ids"],
    max_length=150,
    min_length=50,
    length_penalty=1.5,
    num_beams=6,
    repetition_penalty=2.5,
    early_stopping=True
)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [ ]:
eval_set = dataset["test"].select(range(200))  # or full test set
batch_summaries = [generate_summary(x["Case"]) for x in eval_set]
df = pd.DataFrame({
    "Case": [x["Case"] for x in eval_set],
    "Reference Summary": [x["Summary"] for x in eval_set],
    "Generated Summary": batch_summaries
})
df.to_csv("/content/drive/MyDrive/legal_summaries_output.csv", index=False)
print("📁 Exported batch summaries to legal_summaries_output.csv")

NameError: name 'dataset' is not defined

In [ ]:
rouge = evaluate.load("rouge")
scores = rouge.compute(predictions=batch_summaries, references=[x["Summary"] for x in eval_set])
print("📊 ROUGE Scores:")
for k, v in scores.items():
    print(f"{k}: {v:.4f}")

NameError: name 'batch_summaries' is not defined

In [ ]:
def plot_rouge_scores(rouge_scores):
    keys = list(rouge_scores.keys())
    values = [rouge_scores[k] for k in keys]
    plt.figure(figsize=(8, 4))
    bars = plt.bar(keys, values)
    plt.title("ROUGE Scores for Legal Summarizer")
    plt.ylabel("Score")
    plt.ylim(0, 1)
    for bar, value in zip(bars, values):
        plt.text(bar.get_x() + bar.get_width()/2, value + 0.02, f"{value:.2f}", ha='center')
    plt.tight_layout()
    plt.show()

plot_rouge_scores(scores)

In [ ]:
custom_case_text = """
1. This judgment will dispose of the present application seeking interim injunction against the Defendants and all others acting on their behalf from making, producing, distributing, broadcasting, communicating to the public, adapting, telecasting, exhibiting in theatres, and/or on television and/or online platforms (including any 'OTT platform' or other streaming platform) the impugned film titled "Shamshera", or any part(s) thereof, or any other similar work amounting to infringement of Plaintiff's copyright in his copyrighted script titled 'Kabu na chhadein Khet' (hereinafter referred to as the 'KNCK').

2. Facts to the extent relevant for the present application are that in or around 2006, Plaintiff conceived the idea/concept of a work, which he set out in writing and is stated to be a fictionalised period drama set around the 18th Century. In the year 2008, KNCK, the literary work was manifested/ condensed into a short cinematography film having a runtime of 10 minutes, written and directed by the Plaintiff, wherein the story created by the Plaintiff was set in the 19th Century. In October, 2008, the film was screened at the Spinning Wheel Film Festival, Toronto under title KNCK and the voice over was done by renowned actor Late Shri Om Puri. Plaintiff continued to further flesh out the work and wrote a full-fledged story/script/screenplay running into 40 pages. On 11.05.2009, the concept work was registered with "The Film Writer's Association".

3. Plaintiff asserts that looking out for an appropriate opportunity to pitch his script to a large production house, on 03.02.2016 he approached Defendant No. 5, who was the then Creative Head of Defendant No. 2, so that his work KNCK could be made into a big budget cinematograph film and communicated the main aspects of his work, including that the story was a tale of revenge, oppression, enslavement and freedom involving a small village's rebellion against a mighty foreign invader. Defendant No. 5 responded to the email vide his email dated 04.02.2016 and expressed interest in KNCK and asked the Plaintiff to submit a synopsis and screenplay along with details of Plaintiff's professional background. Defendant No. 5 also stated that Plaintiff should sign a document which Defendant No. 2's legal team would forward in line with their standard turned out that the document was a Non-Disclosure Agreement (hereinafter referred to as the 'NDA'). Defendant No. 5 communicated that a meeting could be set up thereafter, once Defendant No. 2 decided to consider working on KNCK. Subsequently, correspondences were exchanged between Plaintiff and Defendants No. 2 and 5.

4. It is further averred that by email dated 04.02.2016 Plaintiff received a draft NDA from Defendant No. 2's team seeking certain particulars from the Plaintiff to finalise the Agreement. Plaintiff re-worked and further fleshed out the script/screenplay and also populated/printed and signed the NDA. By an email dated 08.02.2016, Plaintiff shared the required particulars with Defendants No. 2 and 5 along with a copy of the signed NDA and by another email dated 03.02.2016 shared the synopsis of KNCK. Responding thereto, Defendant No. 5 by an email dated 23.02.2016, wrote to the Plaintiff that Defendant No. 2's team had read the synopsis and requested for sharing the script/screenplay. Promptly the Plaintiff shared an attachment of the updated screenplay/script with Defendants No. 2 and 5 by email dated 27.02.2016. By email dated 04.03.2016, Defendant No. 5 not only acknowledged reading of the screenplay but also stated that he was interested and suggested next steps including asking the Plaintiff to develop the work further and share a complete script with dialogues to see how it plays out clearly. Plaintiff shared a further re-worked script/screenplay on 12.05.2016 and significantly on 17.05.2016, Defendant No. 5 acknowledged that he had read the complete script and also appreciated the work done. Defendant No. 5 invited the Plaintiff for a meeting at Defendant No. 2's office to discuss the work further.

5. It is further stated that Plaintiff was informed that, an in-house and had offered to read the script shared by the Plaintiff following which the project could be taken ahead. Between August and December, 2016 extensive correspondence/communication took place between the Plaintiff and Defendants No. 5 and 6 including meetings with the creative team of Defendant No. 2, which according to the Plaintiff is evident from communications dated 04.08.2016, 06.08.2016, 08.08.2016 etc. and going upto 25.12.2016. Each of these emails, as asserted, shed light on the fact that the script was read by Defendant No. 6 and even hardcopies of Plaintiff's revised work were shared between Defendants No. 5 and 6. Plaintiff asserts that certain suggestions/inputs were given to the Plaintiff, which were incorporated under the impression that his work would develop into a cinematograph film albeit the changes were mere embellishments and did not detract from the core and/or kernel or the principal ethos of the Plaintiff's work.

6. In January, 2017, however, Defendant No. 5 informed the Plaintiff, to his shock, that he had no intent to work on KNCK as Defendant No. 2 had decided to produce another film of the same genre and Plaintiff was free to pursue other opportunities. As it later transpired the film that was finally released was 'Kesari'. Having apprehension that 'Kesari' was based on Plaintiff's work, he approached this Court in 2019 alleging similarities in the poster of the film. However, on viewing the film 'Kesari', Plaintiff found that only commonality was that the script and the film were of the same genre and therefore, Plaintiff honestly and candidly decided not to pursue the legal action.

7. Plaintiff asserts that in May, 2018, Defendant No. 1 announced its film titled 'Shamshera' produced by Defendant No. 1 with Defendant No. 6 as writer/director and his wife Defendant No. 7 as screenplay writer. Under the guise of confidentiality, several details involving the film were contemporaneously not disclosed.

8. Plaintiff avers that suddenly out of the blue in October, 2020, Defendant No. 6 reached out to the Plaintiff and as the correspondence that ensued would indicate, Defendant No. 6 wanted to touch base with the Plaintiff and asked him to share the latest draft of his work. Defendant No. 6 expressed his intention to go through material again as he was in the process of finalising his next film. Understanding the strategy of Defendant No. 6, Plaintiff decided not to expend any further time on communicating with Defendant No. 6, only to find that on 24.06.2022, trailer of 'Shamshera' was released on YouTube and from the trailer it was clear that the film was based on and/or was an adaptation and/or a substantial reproduction/imitation/copy of Plaintiff's work. This was not only a copyright infringement but a case of breach of confidentiality by the Defendants. Defendants No. 2, 5 and 6 and through them the others had complete access to Plaintiff's work and had parted information to third parties including Defendant No. 1 to enrich themselves unjustly. The access is further evident from the sudden request from Defendant No. 6 in 2020 even after he had moved from Defendant No. 2 to Defendant No. 1.

9. As per the trailer the film was slated for a theatrical release on 22.07.2022. With utmost dispatch Plaintiff wrote an email to Defendants No. 2, 5 and 6 on 25.06.2022 stating that the film was entirely based on KNCK. Defendant No. 6 by an email dated 28.06.2022 denied that there was any substantial similarity between the rival works but there was no denial of access to the entirety of Plaintiff's work. Plaintiff by an email dated 29.06.2022 reiterated that similarities were too many to ignore and sought an audience before screening of the film to allay his apprehensions need to engage further on the subject. This was followed by a legal notice on 06.07.2022 by the Plaintiff, to which there was no response from the concerned Defendants, constraining the Plaintiff to approach this Court.

10. Learned Senior Counsel appearing on behalf of the Plaintiff contended that there were striking similarities between the rival works, which cannot be a mere coincidence. Mere comparison of the Plaintiff's work with the impugned film would show that Plaintiff's work has been plagiarized by the Defendants, who have also acted in gross breach of confidentiality. Defendants' impugned film 'Shamshera' is a substantial reproduction and/or adaptation and/or imitation and/or copying of Plaintiff's work KNCK, details of which have been set out in a comparative table in para 29 of the amended plaint. Defendants have not only infringed Plaintiff's literary work(s) in KNCK by copying it substantially in their film but have also copied the expressions. The rival works when compared, relate to the same genre i.e. period drama. Plaintiff's world/characters include the underdogs with a particular setting in North India. The commonality extends to active protagonist, the legend, the antagonist, animal, birds, boiling hot water etc. used in movie. The core story in the script involves revenge by the son, oppression by foreign invaders, the Afghans, enslavement, freedom, spanning two generations involving a father and his son, based on fictionalised village set in the 1700s/1800s. The script, as it goes, deals with father's legacy and his defiance to fight the foreign invaders which led to his treacherous killing and which proves to be a turning point for the son. The son leads a rebellion against a powerful Afghan army and attacks the antagonist's fortress, exhibiting extraordinary battle skills and valour with the final showdown being face- off with handheld weapons and hand to hand combat between the son and Defendants' film is substantially similar in terms of the genre, the underdogs, setting, active protagonist and antagonist, use of boiling hot water, birds, secret tunnels. The core plot of the film is revenge by the son, oppression by Britishers, enslavement and freedom and a transit between two generations.

11. Much emphasis was laid by learned Senior Counsel for the Plaintiff on the fact that two songs 'Fitoor' and 'Ji Huzoor' are slavishly copied by the Defendants and a detailed comparative table as well as screenshots from the film to support the stand of the Plaintiff were relied on. Compared and contrasted with the stark similarities, the dissimilarities brought forth by the Defendants are trivial and the defence set up against infringement that the similarities, if any, are insignificant and coincidental or those which are common to most films, is a desperate attempt to obfuscate the issue.

12. It was next contended that Defendant No. 6 had clear access to the script/screenplay of the Plaintiff and apparently the same was shared with Defendant No. 1 as a result of which there are so many similarities in the film viz-a-viz the Plaintiff's script. The contradictory stands taken by Defendant No. 6 is itself evidence of the fact that being privy to the script and having access thereto he colluded with Defendant No. 1 and facilitated the making of the impugned film. Post the issuance of legal notice by the Plaintiff, Defendant No. 6 in the reply dated 15.07.2022 stated that he left Defendant No. 2 in May, 2016. Defendant No. 6 admits that he met the Plaintiff in Defendant No. 2's office in August, 2016 and contacted Defendant No. 4 in October, 2016 seeking further improvement in the script to his satisfaction. However, in reply to the injunction application it is stated that he left Defendant No. 2 in 2018 and joined Defendant No. 1 Defendant No. 2 in 2017 and was freelancing till he joined Defendant No. 1 in February, 2018. In fact, Defendant No. 6 perused the script even post- leaving Defendant No. 2 solely to secure his interests and make the film as his own creation. The song Fitoor finds mention in the script and narrative of the Plaintiff, however, the narrative in the script of the Defendants does not portray the manner in which the song is finally presented in the film. This and the fact that two allegedly independent writers, Defendants No. 8 and 9, who had never discussed the script with the Plaintiff, made a substantial copy is reflective of the fact that the common factor was Defendant No. 6, who admittedly had access to the script of the Plaintiff.

13. It was next contended that under Section 13 of the Copyright Act, 1957 (hereinafter referred to as the '1957 Act') copyright subsists in Plaintiff's work and it is sufficient if the Plaintiff proves that Defendants' film is a substantial copy of Plaintiff's work if not a copy of the entire work. By virtue of Section 14 of the 1957 Act all rights in the copyrighted work vest in the Plaintiff as enumerated in the said provision and this is an exclusive right to the exclusion of all others including the Defendants. The acts of the Defendants amount to infringement of Plaintiff's copyright by virtue of a deeming fiction under Section 51 of the 1957 Act.

14. Learned Senior Counsel for the Plaintiff had emphatically drawn the attention of the Court to the comparative tables, alleging that there are multiple similarities in the script of the Plaintiff and the impugned film, but without prejudice to the said argument, had also contended that the test is whether the average audience would recognise the appropriation and invoked the unambiguous concept of de minimis. It was argued that the position of law as it obtains today in the field of idea-expression dichotomy is that even common ideas and themes are protected. In Anil Gupta and Court has held that in modern day, when small screen has taken over the earlier means of mass communication like radio, idea/concept/script of a broadcaster has wider potentiality of capitalising revenue and if that idea is not protected then the person who has conceived the idea, to be translated into a reality TV show, which could be a key to its success, then the channels with their enormous resources would always be in a better position to adopt the idea from any Author and develop it at their own end, leaving the original Author of the concept high and dry. An idea per se has no copyright but if it is developed into a concept fledged with adequate details, the same is registrable under the 1957 Act.

15. Mr. Abhishek Malhotra, learned counsel represented Defendants No. 1, 3, 8 and 9 and Mr. Rajshekhar Rao, learned Senior Counsel addressed arguments on behalf of Defendants No. 6 and 7 and albeit separate written statements were filed but the arguments primarily are common and a common written note has been filed on their behalf.

16. It was contended on behalf of the Defendants that the Plaintiff has failed to make out case for grant of interim injunction, as neither of the ingredients of prima facie case, balance of convenience and irreparable injury, have been made out. To establish a prima facie case of copyright infringement, Plaintiff must establish that Defendants had access to Plaintiff's work and there is substantial similarity in the rival works i.e. an average person upon considering two works would inevitably conclude that Defendants have copied substantial and material portions and the original elements in Plaintiff's work. The two tests are conjunctive and mere access is not enough for the Plaintiff to prove infringement by the Defendants. The reason that substantial similarity is to be established independent of access is because the right conferred in relation to a literary work under Section part thereof and only when a substantial copy is shown, Section 51 of the 1957 Act comes into play. While Defendant No. 6 cannot deny access, however, in the absence of the Plaintiff demonstrating substantial similarity no injunction can be granted. In the past, Plaintiff had sued the Defendants in relation to the film titled 'Kesari' and while there was access, Plaintiff had withdrawn the case after watching the film as the only commonality between Plaintiff's script and 'Kesari' was that they were of same genre.

17. It was urged that the legal requirements required to be fulfilled by the Plaintiff to establish a prima facie case of copyright infringement are crystalized by the Supreme Court in the judgment of R.G. Anand v. Delux Films and Others, (1978) 4 SCC 118. In the principles culled out by the Supreme Court it is clearly elucidated that in order to be actionable, copy must be a substantial and material one, which at once leads to the conclusion that Defendant is guilty of piracy and the surest and safest test to determine violation of copyright is to see if a reader, spectator or viewer after having read or seen both the works is clearly of the opinion and gets an unmistakable impression that the subsequent work appears to be a copy of the original.

18. Plaintiff's argument on substantial similarity is not only flawed but is also inconsistent through the course of hearing. The primary flaw in the argument is that rather than comparing the script with the film to show substantial similarity, Plaintiff took the film, as presented, to be an expression of how his script would be filmed/executed and then proceed to pick individual elements, disjointed from the context to support the allegation of substantial similarity. Furthermore, erroneously the Plaintiff seeks to claim rights over factual/thematic/ideation such as period drama, where father looks like the son and takes revenge. Interestingly, while initially Plaintiff alleged that the entire film was a copy of the script, the argument over the course of hearing changed to a substantial copy and finally emphasis was laid on the two songs and their identity. While this inconsistent stand cannot be permitted, even otherwise none of the three aspects alleged to be copied substantially, can establish infringement by the Defendants, even prima facie. Insofar as the period drama is concerned, Plaintiff himself admits that the script is inspired by 'Bahubali' and in any case there are many other films that have been period dramas. Countless films, such as, Kranti, Kesari, Mard, Braveheart, etc. have been made on story of suppression by foreign invaders and revenge. The aspect of revenge in Plaintiff's script is nothing to do with the son taking revenge of the father and there are only stray references to the protagonist's father in Plaintiff's script. It is also to be noted that there are multiple films which have presented this thematic idea and Plaintiff cannot claim monopoly. As an illustration, movies like Zanjeer, Agneepath, Aakhree Rasta, etc. were cited on behalf of the Defendants. Similarly, use of burning oil to scald has been used in multiple stories/films including Slumdog Millionaire etc. and the secret under water tunnels are also featuring in several films such as The Tunnel, The Great Escape and so on. Plaintiff claims similarity in the theme on the anvil of the antagonists in both works being barbaric foreign invaders, who plunder the village and the protagonists fighting for freedom from this terror. A 'theme' cannot be accorded copyright protection as per the settled law and additionally, there are various films which have typified this theme, such as Kesari and is best a 'stock'. Even otherwise the themes are materially different and distinct in the rival works. While the village of Noorpur is central to the theme and story of KNCK, as possibly the only such village that is subject matter of the film. Instead, the film refers to a tribe of erstwhile warriors who have been shunned by the society as lower caste and have retreated to the jungle, where the only way of survival is by looting the upper castes. The upper caste locals bribe the British, who are not portrayed as barbaric invaders, to get rid of the looting menace of the Khameran tribe. The central barbaric character is not a foreigner but Shuddh Singh, an upper caste Police officer, who tricks the Khamerans into slavery.

19. It was contended that the theme of death of a father and the son carrying forward his legacy is also not copyrightable. Multiple films such as Agneepath, Braveheart are based on this theme. Character of Shamsher in KNCK is not relevant or central to the script, which revolves around resistance of Kartar and his villagers against the marauding Afghans. Kartar is not carrying forward his father's legacy. The film revolves around the treaty signed between the British and Shamshera, a price that Khamerans had to pay for their freedom and the task which could not be completed by the father due to his killing, is completed by the son Balli. There are differences between the lead characters and their respective roles including the climax. There cannot be a copyright infringement in concepts or ideas which emerge from the scenes in the script, besides the fact that they are common to several earlier films or are based in history.

20. It was argued that much emphasis was laid by the Plaintiff on the fact that there are certain scenes in the film such as back-stabbing in the climax as well as in the description of the song sequence for the song Fitoor including the scene of the protagonist putting on a ghaghra for the song Ji Huzoor. The over-emphasis is misplaced, since unlike a script, which is an individual and isolated process, film making is collaborative, music composers, choreographers, art directors, producers etc. Thus, even when a script/screen play is said to be 'locked', it is always subject to modifications on account of various factors such as locations, weather, availability of props, animals etc.

21. It was contended that insofar as the two songs are concerned, which was really the bone of contention to begin with, this comparison itself is incorrect and impermissible in law and contrary to the ratio of the judgment in R.G. Anand (supra), which holds that works must be compared as a whole and not by picking and choosing parts of one to compare with parts of another. The description of the song by the Plaintiff is, in any event, not protectable since it consists only of an idea/theme of two lovers engaged in a sensuous song sequence in the backdrop of snow and wrapped in a red dari. Perhaps the only part protectable is the use of red dari, which significantly was not used by the Defendants. The scene description in the Plaintiff's script is not original and unprotectable since the same has been used in numerous films such as Mohra, Chandni etc. Moreover, even on the touchstone of qualitative comparison, the song sequences are not a substantial part of Plaintiff's work, since four out of five versions of the script do not have this description of the song. There is no co-relation between the description in the script and the screenplay of the song in the Defendants' film as is demonstrable from a mere viewing of the song. Defendants' song contains a more detailed expression and bigger canvass of a generic love sequence, typical to a lot of Bollywood film songs, where on a plateau with sand, the lead actor and actress make love with each other, finally leading to their marriage.

22. Plaintiff has repeatedly asserted that there is stark similarity in the use of ghaghra in a song sequence Ji Huzoor. According to the Plaintiff's dancer swirls her ghaghra and various dancers emerge from under the ghaghra. However, contrasted with this, in the film there is no swirling of the ghaghra to show emergence of children. The song sequence has the protagonist dancing with kids and as part of the mischievous tilt to the song sequence, the man jumps off a portion of a living space and laches on to the clothing to cushion his fall, none of which is present in the Plaintiff's script. There cannot be a copyright in an idea of using a certain piece of clothing, which is common place and nor in the idea of clothing being swirled or emergence of people from under it.

23. The entire exercise of infringement analysis by the Plaintiff is convoluted and unknown to law. Plaintiff is required to first identify his work, especially the qualitative substantive portions and then demonstrate how those have been substantially copied by the Defendants as an expression, in the film. Instead, Plaintiff has taken Defendants' film as the starting point, instead of his script as the base and juxtaposed different and out of context situations from his script to distort and fit the same into Defendants' film. Interestingly, there is no dispute or denial in replication to the paras of the written statement where Defendants have brought forth a summary of scene by scene flow of Plaintiff's five scripts compared with scene by scene flow of the film. Plaintiff has himself acknowledged that his story is based on and inspired by earlier films such as 'Bahubali' and different parts of the script referred to song sequences such as 'kajra mohabbatwala' or 'mere haathon mein nau nau churiyan' evidencing that inspiration from existing works is common place in the film industry.

24. It was argued that the ideas, expressions, storytelling, storyline, characters, caste issues, are different in the rival works and thus mere access to the script albeit admitted by the Defendants, would not justify a similarity in the two works. Plaintiff relied on the judgment in Anil Gupta (supra), to argue that an injunction order was granted in respect of use of an idea of an age-old tradition such as Swayamwar and therefore, generic elements can be monopolized and protected under the Act. This is a fallacious argument as in the said judgment the injunction was granted not to protect the concept of Swayamwar but the manner in which it was expressed or portrayed by the Plaintiffs and another aspect that went against the Defendants was their refusal to provide details of their proposed expression of Swayamwar. In the present case, the ideas, expressions etc. all being different, the judgment is inapplicable.

25. It was also urged that Plaintiff has misdirected itself in arguing the principle of de minimis non curat lex. The said principle is advanced as an argument in defence of a fair use and while taking such a defence there is an admission of direct copying of parts of the copyrighted work. In other words, the defence is that notwithstanding the admitted copying, the same is so miniscule in the larger scheme of things that Court ought not to waste its time in adjudicating the matter. In the present case, Defendants' position clearly is that there is no copying at all of the Plaintiff's work and thus, the defence of de minimis does not arise. Plaintiff has also argued, relying upon Section 14(d)(i)(B) that even a small amount of copying such as a song, photograph of a film, amounts to copyright infringement and even if the Defendants have quantitatively taken a small portion of Plaintiff's work i.e. the song sequence alone, it would constitute copyright infringement. Plaintiff claims that his right under Section 14(a)(iv) has been infringed i.e. right to make a cinematograph film based on the script which is a literary work and therefore, he cannot now make a film on his own script since the Defendants have made the film substantially similar to the script. Albeit this submission is not pleaded, even otherwise, Plaintiff cannot use an analogy from a provision of law relating to a different category of work.

26. Plaintiff has failed to make out a prima facie case of copyright infringement. In terms of the settled principles of grant of interim injunction, the standard required to be considered for balance of convenience and irreparable injury has been aptly elucidated in the judgment in the case of Wander Ltd. and Another v. Antox India P. Ltd., 1990 Supp SCC 727, where the Court held that corresponding hardships/ inconveniences likely to be caused to either party in the event the injunction is or is not granted have to be considered by Courts. It is Plaintiff's pleaded case that he was aware of the film since May, 2018 and in October, 2020, Defendant No. 6 reached out to him to ask for the script. Despite this, Plaintiff waited till 03 days prior to the proposed release date to file the present suit. Additionally, film has been released in theatres and on the OTT platform and is already in the public domain. Grave hardship shall be caused to the Defendants if the satellite release is stalled or Defendants are directed to take the film of the various platforms, as multiple agreements have been signed with third parties in this regard. On the other hand, in case the Court comes to a conclusion, post-trial that Plaintiff has been wronged, he can always be compensated in terms of money, but the injury caused to the Defendants shall be irreparable and irretrievable, if injunction is granted. Moreover, Plaintiff's work has never been published and it is always open to him to make a separate film based on the script authored by him.

27. Arguing on behalf of the Plaintiff, in rejoinder, it was contended that it is incorrect for the Defendants to argue that the script of the Plaintiff and the film are different in ideas and/or expressions, screen play, plot etc. The the central kernel/core/ethos of Plaintiff's work have been blatantly copied. It is not the manner in which the film is presented but the basic ploy or plot and the storyline are the focal points of comparison to see if there is a substantial copy. Plaintiff pleads that the basic plot and storyline of the script has been copied and that too substantially, thereby infringing Plaintiff's copyright in the script. Defendant No. 6's statement that film is a periodic film and not a historical drama makes it evident that the film has taken several inputs from the script of the Plaintiff, which was admittedly accessible to him. It is a settled position of law that plots and storylines in the script are protected under the copyright law and mere reduction in figures and modifications introduced in some characters or clothes or colours are too trivial to escape violations of copyright.

28. The contention was that in R.G. Anand (supra), the Supreme Court has held that the question is not whether the alleged infringer could have obtained the same information by going to the same source used by the Plaintiff in his work, but whether he did in fact go to the same source and did his own independent research. In other words, whether he made an independent production or made a substantial and unfair use of Plaintiff's work. It was also held that in an attempt to show that he is not guilty of copyright infringement, it is always possible for a person intending to take advantage of the intellectual effort and labour of another to so develop his own product that it covers a wider field than the earlier product and introduce changes in the common area in order to disguise the attempt at plagiarism. In Kapil Chopra v. Kunal Deshmukh and Others, 2012 SCC OnLine Bom 1302, the Bombay High Court held that the Court will look strictly at not only the blatant examples of copyright but also at reprehensible attempts at colourable imitation and once the Court comes to genesis in the idea, a clear case of grant of injunction is made out. In Harman Pictures N.V. v. Osborne and Others, (1967) 1 WLR 723, it was held that similarities of incidences and situations afford prima facie evidence of copying and in absence of any explanation by the Defendant regarding the sources, Plaintiffs must succeed. In Urmi Juvekar Chiang v. Global Broadcast News Ltd. and Anr., 2007 SCC OnLine Bom 471, Bombay High Court held that the Court needs to ascertain whether the work of the Defendants is similar in material and substantial aspects with that of the Plaintiff and referred to the observations in an earlier judgment of the Bombay High Court in Zee Telefilms Ltd. v. Sundial Communications Pvt. Ltd. and Others, 2003 SCC OnLine Bom 344, where it was held that it is enough that substantial parts are lifted and no play-right can excuse wrong for showing how much of his work he did not pirate. In Twentieth Century Fox Film Corporation v. Sohail Maklai Entertainment Pvt. Ltd. and Another, 2010 SCC OnLine Bom 1577, Bombay High Court held that it is the quality of the copied work and not the quantity that would determine infringement of the work. Reliance was placed on a paragraph from Halsbury Law of England, Fourth Edition, 2006, Volume 9(2), Page 316, stating that sufficient objective similarity between two works and some casual connection is enough to establish copyright infringement.

29. Assuming without admitting that what has been copied are stock elements, however, it cannot be a matter of sheer coincidence that so many stock elements are common between the script and the film. It is a matter of fact that Plaintiff's script is a period drama containing underdogs, set in North India, oppressed by foreign invaders, having an active protagonist, a legend and an antagonist and after all this was shared with Defendant No. and 'Afghans to Britishers' cannot give any defence to the Defendants. In India TV Independent News Service Pvt. Ltd. and Others v. Yashraj Films Pvt. Ltd., 2012 SCC OnLine Del 4298, a Division Bench of this Court and in Shemaroo Entertainment Limited v. News Nation Network Private Limited, 2022 SCC OnLine Bom 930, the Bombay High Court have clearly enunciated that it is the qualitative element which matters and even if the exploitation is miniscule it cannot come to the aid of an infringer. The factors laid down in India TV Independent News Service Pvt. Ltd. (supra), commonly considered by the Courts in applying de minimums are: (i) size and type of harm; (ii) cost of adjudication; (iii) purpose of the violated legal obligation; (iv) effect on legal rights of third parties; and (v) intent of the wrong doer. In Kapil Chopra (supra), the Bombay High Court noted that injunction before the release of the film is rarely granted as Courts at that time do not have benefit of examining the script and in this scenario, if there is a case of plagiarism, writer is left with no remedy at all. Writers of the film are not treated with the same dignity and respect which they deserve. It was thus prayed that injunction be granted directing the Defendants to take the film off all the platforms.

30. I have heard the respective learned Senior counsels for the Plaintiff and Defendants No.6 & 7 as well as other counsels for the remaining Defendants and examined their rival contentions.

31. The heart of the dispute in the present application lies in the rival works: Plaintiff's script 'KNCK' and Defendants' film 'Shamshera'. Indisputably, Plaintiff has copyright in the script being the original author of the script KNCK. Before proceeding to examine rival claims of the parties on merits, it would be crucial to closely look at the judicial precedents on the law of copyright, especially in the field of judicial scrutiny at this interlocutory stage. In order to succeed at this stage, Plaintiff must establish a prima facie case of infringement of the copyright albeit with a caveat that Plaintiff need not show that he has chances of succeeding at the end of the trial. It is settled that the remedy is discretionary and the discretion has to be exercised keeping in mind also the remaining two parameters of the three trinity principles i.e. balance of convenience and irreparable injury and harm.

32. The complexed question before this Court is whether the Defendants have copied the idea and/or the expression of the idea of Plaintiff's work substantially in the impugned film, making themselves liable for copyright infringement. Idea-expression dichotomy is fundamental to the law of copyright and there is a sea of judicial precedence on this. The landmark judgment on this dichotomy, as is well known, is by the Supreme Court in the case of R.G. Anand (supra). Going back in point of time, the earliest formulation of this dichotomy was by the English Court in the case of Hollinrake v. Truswell, 1894 Ch. 420, where it was ruled that copyright does not extend to ideas or schemes or methods and is confined to their expressions and if expression is not copied, copyright is not infringed. Few years later in 1954, the US Supreme Court in Mazer v. Stein, 347 U.S. 201, 218 (1954), reiterated that a copyright is no exclusive right to the art disclosed and protection is given only to expression of the idea and not the idea itself. In R.G. Anand (supra), the Supreme Court referred to an earlier judgment in K.R. Venugopala Sarma v. Sangu Ganesan, 1972 Cri LJ 1098, which was a case of infringement of copyright in a picture and it was held that the degree of resemblance between two pictures, which is to be judged by the eye, must be such that the person looking at the Respondent's picture gets a suggestion that it is Appellant's work and one former picture finds place in the reproduction. The Supreme Court then observed that an idea, principle, theme, subject matter or historical or legendry facts being common property cannot be subject matter of copyright. It is always open to any person to choose an idea as a subject matter and develop it in his own manner and give expression to the idea by treating it differently from others. Significantly, it was also observed that where two writers write on the same subject, similarities are bound to occur because the central idea of both is the same and similarities or coincidences by themselves cannot lead to an irresistible inference of plagiarism or piracy. The Supreme Court in R.G. Anand (supra), elucidated the following principles on idea-expression dichotomy:-

"45. Thus, the position appears to be that an idea, principle, theme, or subject-matter or historical or legendary facts being common property cannot be the subject-matter of copyright of a particular person. It is always open to any person to choose an idea as a subject-matter and develop it in his own manner and give expression to the idea by treating it differently from others. Where two writers write on the same subject similarities are bound to occur because the central idea of both are the same but the similarities or coincidences by themselves cannot lead to an irresistible inference of plagiarism or piracy. Take for instance the great poet and dramatist Shakespeare most of whose plays are based on Greek- Roman and British mythology or legendary stories like Merchant of Venice, Hamlet, Romeo and Juliet, Julius Ceasar etc. But the treatment of the subject by Shakespeare in each of his dramas is so fresh, so different, so full of poetic exuberance elegance and erudition and so novel in character as a result of which the end product becomes an original in itself. In fact, the power and passion of his expression, the uniqueness, eloquence and excellence of his style and pathos and bathos of the dramas become peculiar to Shakespeare and leaves precious little of the original theme adopted by him. It will thus be preposterous to level a charge of plagiarism against the great playwright. In fact, throughout his original thinking, ability and incessant labour Shakespeare has converted an old idea into a new one, so that each of his dramas constitute a masterpiece of English literature. It has been rightly said that "every drama of Shakespeare is an extended metaphor". Thus, the fundamental fact which has to be determined where a charge of violation of the copyright is made by the plaintiff against the defendant is to determine whether or not the defendant not only adopted the idea of the copyrighted work but has also adopted the manner, arrangement, situation to situation, scene to scene Indeed, if on a perusal of the copyrighted work the defendant's work appears to be a transparent rephrasing or a copy of a substantial and material part of the original, the charge of plagiarism must stand proved. Care however must be taken to see whether the defendant has merely disguised piracy or has actually reproduced the original in different form, different tone, different tenor so as to infuse a new life into the idea of the copyrighted work adapted by him. In the latter case there is no violation of the copyright.
46. Thus, on a careful consideration and elucidation of the various authorities and the case law on the subject discussed above, the following propositions emerge:
1. There can be no copyright in an idea, subject-matter, themes, plots or historical or legendary facts and violation of the copyright in such cases is confined to the form, manner and arrangement and expression of the idea by the author of the copyrighted work.
2. Where the same idea is being developed in a different manner, it is manifest that the source being common, similarities are bound to occur. In such a case the courts should determine whether or not the similarities are on fundamental or substantial aspects of the mode of expression adopted in the copyrighted work. If the defendant's work is nothing but a literal imitation of the copyrighted work with some variations here and there it would amount to violation of the copyright. In other words, in order to be actionable the copy must be a substantial and material one which at once leads to the conclusion that the defendant is guilty of an act of piracy.
3. One of the surest and the safest test to determine whether or not there has been a violation of copyright is to see if the reader, spectator or the viewer after having read or seen both the works is clearly of the opinion and gets an unmistakable impression that the subsequent work appears to be a copy of the original.
4. Where the theme is the same but is presented and treated differently so that the subsequent work becomes a completely new work, no question of violation of copyright arises.
5. Where however apart from the similarities appearing in the two works there are also material and broad dissimilarities which negative the intention to copy the original and the coincidences appearing in the two works are clearly incidental no infringement of the copyright comes into existence.
6. As a violation of copyright amounts to an act of piracy it must be proved by clear and cogent evidence after applying the various tests laid down by the case-law discussed above.
7. Where however the question is of the violation of the copyright of stage play by a film producer or a director the task of the plaintiff becomes more difficult to prove piracy. It is manifest that unlike a stage play a film has a much broader prospective, wider field and a bigger background where the defendants can by introducing a variety of incidents give a colour and complexion different from the manner in which the copyrighted work has expressed the idea. Even so, if the viewer after seeing the film gets a totality of impression that the film is by and large a copy of the original play, violation of the copyright may be said to be proved."
33. Coming now to some of the provisions of the 1957 Act, Section 13 thereof provides that subject to the said Section and other provisions of the said Act, copyright shall subsist throughout India in original, literary, dramatic, musical and artistic works, cinematograph films and sound recording. 'Work' is defined in Section 2(y), to include literary, dramatic, musical or artistic work, cinematograph film and sound recording. Section 14 defines copyright to be an exclusive right to do or authorize the doing of the acts in respect of a work or any substantial part thereof as provided in Clauses (i) to (vii) therein. In Eastern Book Company and Others v. D.B. Modak and Another, (2008) 1 SCC 1, the Supreme Court held that the word 'original' does not mean that the work must be the expression of original or inventive thought. Copyright Acts are not concerned with originality of ideas, but with the expression of thought and the 1957 Act does not require that the expression must be original or novel, but that the work must not be copied from another work i.e. it should originate from the author. Relevant paragraphs are as follows:-

"32. The word "original" does not mean that the work must be the expression of original or inventive thought. The Copyright Acts are not concerned with the originality of ideas, but with the expression of thought, and in the case of literary work, with the expression of thought in print or writing. The originality which is required relates to the expression of the thought. But the Act does not require that the expression must be in an original or novel form, but that the work must not be copied from another work--that it should originate from the author; and as regards compilation, originality is a matter of degree depending on the amount of skill, judgment or labour that has been involved in making the compilation. The words "literary work" cover work which is expressed in print or writing irrespective of the question whether the quality or style is high. The commonplace matter put together or arranged without the selection will not be entitled to copyright. The word "original" does not demand original or inventive thought, but only that the work should not be copied but should originate from the author. In deciding, therefore, whether a work in the nature of a compilation is original, it is wrong to consider individual parts of it apart from the whole. For many compilations have nothing original in their parts, yet the sum total of the compilation may be original. In such cases the courts have looked to see whether the compilation of the unoriginal material called for work or skill or expense. If it did, it is entitled to be considered original and to be protected against those who wish to steal the fruits of the work or skill or expense by copying it without taking the trouble to compile it themselves. In each case, it is a question of degree whether the labour or skill or ingenuity or expense involved in the compilation is sufficient to warrant a claim to originality in a compilation.
33. While considering the question whether the copyright protection is available to the work created as a whole or the fragment of the work would be considered piecemeal and individually apart from the whole, the House of Lords said as under: (Ladbroke case [(1964) 1 WLR 273 :
(1964) 1 All ER 465 (HL)] , All ER p. 469 B-D) "... One test may be whether the part which he has taken is novel or striking, or is merely a commonplace arrangement of ordinary words or well-known data. So it may sometimes be a convenient short cut to ask whether the part taken could by itself be the subject of copyright. But, in my view, that is only a short cut, and the more correct approach is first to determine whether the plaintiff's work as a whole is 'original' and protected by copyright, and then to inquire whether the part taken by the defendant is substantial.
A wrong result can easily be reached if one begins by dissecting the plaintiff's work and asking, could Section A be the subject of copyright if it stood by itself, could Section B be protected if it stood by itself, and so on. To my mind, it does not follow that, because the fragments taken separately would not be copyright, therefore the whole cannot be."

34. The law is no longer res integra that to be actionable the copy of the Plaintiff's work by the Defendant must be substantial and the surest and safest test to determine whether there is a substantial copy is to see if the reader, spectator or viewer after having read or seen both the works is clearly of the opinion and gets and unmistakeable expression that the subsequent work appears to be a copy of the original. Where the theme is the same and is presented differently, no question of violation of copyright arises. In R.G. Anand (supra), the Supreme Court perspicuously observed that the Court should determine whether or not similarities are on fundamental or substantial aspects of the mode of expression adopted in the copyrighted work. In other words, the copy must be a substantial and material one, which at once leads to the conclusion of piracy by the Defendant. Earlier, the Judicial Committee in the case of Macmillan & Company Limited v. K. and J. Cooper [51 IA 109: AIR 1924 PC 75], pointing out the essential ingredients of copyright infringement, observed that to constitute piracy of a copyright, it must be shown that the original has been either substantially copied or to be so imitated as to be a mere evasion of the copyright. This leads to an ancillary but an important question on what would be the test of substantial similarity. From the conspectus of various judicial pronouncements, it emerges that the test of substantial similarity entails comparison of the rival works in their entirety and it is not permissible to split or dissect the works into parts. [Ref.:

Twentieth Century Fox Film Corporation (supra); Star India Private Limited v. Leo Burnett (India) Private Limited, 2002 SCC OnLine Bom 942; and Barbara Taylor Bradford & Anr. v. Sahara Media Entertainment Ltd. & Ors., 2003 SCC OnLine Cal 323]. It is relevant to note the observations in this context of Bombay High Court in Mansoob Haider v. Yashraj Films Pvt. Ltd. and Others, 2014 SCC OnLine Bom 652 that commonality must not be in dissected and disparate elements but in what lies at the heart and core of the competing works. Plaintiff has to show that an average person reading the two scripts or reading his script and seeing the movie would inevitably conclude that the film was a copy, lifted from the Plaintiff's work. Relevant para of the judgement is as under:-
"28. It does not seem to me in the least clear what it is precisely that the Plaintiff claims to have a monopoly on. Is it the fact of there being a vanishing trick as a magic act? Of a magician with twins? Of one of the twins having an identifying mark? None of these are original conceptualizations. Mr. Dhond is correct, therefore, when he asks what is it in which the Plaintiff claims such 'originality'? It cannot be the theme of twins, one with a distinguishing mark. This is as old as the hills, even in Bollywood. By the Plaintiff's own admission, it is not in the magician or his vanishing trick. It is not in the concept of murders replicating fiction. There can be no copyright in elements that constitute scène à faire. Remove all these elements, Mr. Dhond says, and nothing 'original' remains as the source from which the film could be said to be have been copied. This, I believe, is a compelling argument. The burden on the Plaintiff was rather more. He had to show commonality not in dissected and disparate elements but in what lies at the heart and core of the competing works. He had to show that an average person reading the two scripts or reading his script and seeing the movie would inevitably conclude that the film was a copy, lifted from the Plaintiff's work. That is simply not possible."
35. I may first examine the exposition of law in respect of the contention of the Plaintiff that Defendant No.5 and later other Defendants had access to his script, which be it noted, is admitted by the Defendants today, on a factual note. This issue came up before the Bombay High Court in Mansoob Haider (supra). Relying on the judgment in R.G. Anand (supra), the Bombay High Court observed that there is a singular test for copyright infringement and which is that if we remove all scenes to which no originality attaches, such as all "scène à faire", then are the two works still substantially similar and has the subsequent work copied substantial part of the earlier work. The Court carved out three crucial questions/legal tests as follows: (a) has the Plaintiff proved that the defendant had access to his work? (b) on considering the two works, would an ordinary person inevitably conclude that the defendant had copied the plaintiff's work? (the subjective or intrinsic test); and (c) is there a substantial and material overlapping or commonality of the original elements in the plaintiff's work? The Court thereafter held as under:-

"47. Even if a plaintiff fails on the first question, he may yet succeed on the second and third questions. But if he fails on the second question also, then I do not see how he can possibly succeed on the third alone. But that may arise in another matter. In this case, in my view, the present Plaintiff fails on all three counts. Indeed, his case is not even based on the second question, but only on the first and his own variation of the third : that Defendants Nos. 1 to 3 had access and that there are common elements, even if these are not shown to be entirely unique and some of which are later given up as being original (the magic trick) or demonstrated to be untrue (being set in a foreign locale). The Plaintiff's variation on the third question is a sort of reductio ad absurdum : a vivisection of individual elements, a false and misleading juxtaposition of these, and, on that basis, to 'round up the usual suspects' and invite a finding of infringement. If these elements, however and wherever placed, are in support of an entirely different premise and story line, there can be no copying, no piracy and no infringement."
36. Therefore, what follows from the above observation is that access alone cannot be a ground to come to a conclusion that there is copyright infringement and Plaintiff will require to establish substantial similarity albeit even if the Plaintiff fails on the question of access, he may succeed if substantial similarity is established but the converse cannot be accepted. With these principles in the backdrop, I may now proceed to examine the case on its facts and the charge of violation and infringement of copyright, levelled by the Plaintiff against the Defendants.

37. The trigger of filing the present suit was the release of the trailer of the impugned film on YouTube on 24.06.2022, whereafter Plaintiff sent an e-mail seeking purview of the film followed by a legal notice on 06.07.2022, when there was no response. The suit was essentially a qua timet action filed prior to the theatrical release scheduled on 22.07.2022. Since the suit was filed on the basis of a trailer, the plaint contained limited pleadings as well as a comparative of the Plaintiff's work in KNCK and Defendants' impugned film. Right was reserved in the plaint to file a detailed comparison after viewing the actual film. On 02.08.2022, while the Platforms was continued, Defendants had undertaken to provide to Plaintiff's counsel and the Court a link on which the encrypted version of the movie could be viewed. It is an undisputed fact that the impugned film was released in the theatres on 22.07.2022, as no injunction was granted by the Court. Subsequently, Defendants No. 1, 3, 8 and 9 prayed that Defendant No. 1 be permitted to release the film on various OTT Platforms as the release was scheduled on 19.08.2022 on the ground that failure to do so would amount to breach by the said Defendants of the contractual obligations qua third parties and irreparable injury shall be caused to Defendant No. 1 while Plaintiff could always be compensated in terms of money if he succeeds. Considering the totality of circumstances and the fact that the film had been released in theatres, Court permitted Defendant No. 1 to release the film on OTT Platforms. However, to balance equities, Defendant No. 1 was directed to deposit a sum of Rs.1 crore with the Registrar General of this Court. This was, needless to state, without prejudice to the rights of either party. After the movie was released and viewed by the Plaintiff, amended plaint was filed in which a detailed comparative was drawn in a tabular form bringing forth the similarities between the script of the Plaintiff and the film of the Defendants. Defendant No. 1 has also brought forth in the written statement a detailed comparative between the script and the film pointing out the similarities and the differences.

38. Having carefully perused the script of the Plaintiff and having viewed the movie including the detailed comparative tables made by both parties on which they have spent considerable labour, I would first give a summary of the scripts of the Plaintiff. Be it noted that admittedly the script of the Plaintiff has five versions and this too was one of the serious

39. Coming to the scripts of the Plaintiff and seen holistically, the theme and concept of Plaintiff's work titled KNCK is a tale of valour of the Sikh community set in the backdrop of 18th century with a particular geographical location i.e. the border of India with Afghan territories. The script focusses on a tribute to the community for its role played in defending the onslaught of Afghan invaders since the geographical location made them the first target of the Afghan invasions. The story draws inspiration from the fact that the Sikhs were instrumental in stopping repeated Afghan attacks including the Battle of Sialkot in 1763 when the Afghan army was defeated and focusses on the role of inhabitants of a single village, Noorpur and the lead character Kartar.

40. The script begins with a narrative of the rise of foreign invaders, Afghans and Mughals with a focus on the villages of Punjab. An Afghan leader, Rashid Khan attempts to take over the villages in Punjab and during one such raid, comes across rebellious Guddaan in village Dittopur. When Rashid Khan threatens to wipe out the village in the absence of the villagers unable to answer his questions, Guddaan insults Rashid Khan in a verbal battle. Rashid Khan takes a liking for Guddaan and takes her as a prisoner along with other girls from the village. Kartar is a rebellion in another village of Punjab, Noorpur, and is trained by the Nihaang army from his childhood. The character is portrayed as an ill-tempered, regular drinker, with bitterness against his family members i.e., mother, grandfather and stepfather and as the plot unfolds, it is revealed that his stepfather was the uncle of Kartar who had married his mother after the death of his biological father. Kartar always disliked his stepfather while the latter lived in the hope that relationships may improve. Kartar has a group of friends namely, Ronak, Bhulla, Dhartidhak (a wrestler) and Jaangu (a pro at slingshot) and enjoys playing sports such as wrestling and tug of war. Kartar is a strong contender in the sporting events and enjoys a dominating/ decision making position amongst his friends, much to the dismay of one of his friends Dharam Singh. Kartar and his friends raid the Afghan convoy camping at the riverside of Noorpur village and using their arms and weapons to secure the supplies of grains, dairy, poultry etc. the raids are intentionally conducted in the night to take the Afghans by surprise and affect their preparedness for the next day. Afghans including Rashid Khan and his brother Dilawar are engaged in revelry on the night of rebellion led by Kartar. While camping, Rashid Khan orders Guddaan, whom he had taken captive to be sent to his tent. Guddaan is furious and ready to kill herself rather than be given to Rashid Khan. Though Rashid tries to force himself on her, Guddaan manages to attack him with a dagger as he gets distracted due to the commotion outside on account of the raid by Kartar and his friends.

41. In the fight that follows, Kartar and his friends slaughter the sleeping Afghan soldiers and manage to get hold of the loot and also free the girls taken as prisoners including Guddaan. Guddaan is, however, not accepted in the village as according to the villagers she was impure after being taken captive by the Afghans. Guddaan is infuriated on the allegations and insults levelled on her by her relatives. Realising that Guddaan could not be abandoned, Kartar's friends disguise her as a boy named, 'Guddu' and take her back to their village, Noorpur where she is introduced to everyone including Kartar who, impressed by Guddu, takes Guddu under his wing. Kartar's family accepts Guddu as a part of the family and Guddu also begins to understand Kartar's actions in the light of his past and sees him in a new light.

42. Guddu and Kartar become good friends and Guddu's skill in sports continues to impress Kartar. Rashid Khan on the other hand continues to search for Guddaan and arrives at Noorpur but is unable to find her. Meanwhile, mistakenly, Guddu's identity becomes known to Kartar when he goes in search of Guddu and chances upon her in the washroom. Kartar is furious on his friends for having brought Guddaan back to village as a boy but eventually mesmerized by her beauty. Kartar demands that either he or Guddaan will stay in the house and eventually, leaves the house in anger. Guddaan learns more of Kartar's past from his mother who confides in her including how Kartar's father Shamsher died trying to save a helpless girl from the clutches of the Afghan singularly. On returning home, thoroughly drunk, Kartar takes Guddaan on a horse to return her back to her relatives, however, during the journey, he has a change of heart when Guddaan convinces him of the blessings he had in the form of his family. Kartar inquires about his father from an old guard in Noorpur who reminisces about Shamsher as being a great hero contrary to what Kartar thought of him. As a changed person for good, Kartar returns to the village and changes his way of life and his family is overwhelmed to see this change in his personality. Dharam Singh, his friend, is however agitated by this change. Kartar and Guddaan fall in love with each other.

43. The next plot is again about Rashid Khan inquiring desperately for Guddaan, in furtherance of which he sets up a camp near Noorpur. Dharam Singh sees this as an opportunity to attack the Afghans, without the authority of Kartar and finally, he and his gang are tortured and killed by the Afghans. Dilawar learns about Guddaan's presence in the village and attacks the villagers in a heated pursuit for the girls. Kartar and his friends come across Dharam and Bheeshan's bodies hanging from poles outside Kartar and his friends overpower Dilawar and his soldiers and tie them to the walls of the haveli as a negotiation strategy, should Rashid Khan attack the village. Dilawar is beheaded by Kartar in a fit of anger. This triggers a battle between Rashid Khan and the villagers and Kartar seeks aid from the Nihaang Army.

44. Before the final battle, Kartar reconciles his differences with the family. While Afghan soldiers tried to climb a wall to enter the village, they find themselves covered in boiling oil and faced stone pelting at the hands of Jangu along with other children of the village. Unable to gain access to the village, Rashid Khan enters Noorpur from the village community well. In the middle of the battle, Rashid Khan tries to negotiate with Kartar that the entire village will be spared if he hands over Guddaan, which offer is declined. Rashid Khan takes Kartar's father as a hostage, who expresses his readiness to die in exchange for Guddaan. Nihaang army arrives at the right time and finally the one-to-one combat between Rashid Khand and Kartar ends in the death of Rashid Khan. The final battle results in the victory of the villagers of Noorpur and the screenplay ends with a description of the formation of the courageous Sikh empire under the leadership of Maharaja Ranjit Singh.

45. The Defendants' film, on the other hand, is a work of fiction based in the 19th century India and the theme is caste division and plight of the lower caste communities constantly tortured and harassed by people from the higher caste. The caste in question in the Film is titled "KHAMERAN". The story begins with a depiction of territorial boundaries that existed in Rajputana State during 1570s. The Khameran tribe were known for their valour and courage displayed alongside the Rajputs in the battles against the Mughals. Despite their best efforts, Rajputs were decimated by the Khamerans did not have any army or a leader and begged for shelter in Kaza, in the northern Province of India now Rajasthan. Shunned away by the upper caste, Khamerans lived in the forests surrounding Kaza and survived on looting the upper castes. The story depicts the era of 1871when the looting activities of Khamerans were at the peak and leadership was in the hands of Shamshera, a powerful and good man. Tired of the looting activities of the Khamerans, the upper castes approached the British commander in Kaza and sought help offering gold as compensation as price for the help. Shuddh Singh, Antagonist, who was an Indian police officer working under the British, Daroga and belonged to the upper castes offered to help. In the battle that pursued, both sides lost equal number of people and Shamshera captures a senior British officer. Shuddh Singh pretends to offer peace by asking the Khamerans to surrender and stop looting, in exchange of which they would get a permanent residence and return to a normal life. Shamshera and his tribe of Khamerans agreed to accept and surrender and are taken to Kaza Fort. Upon entry, when the gates of the Fort are shut, Shamshera realizes his mistake that the offer of settlement was only a ploy to get them captive. The enslaved Khamerans are bound by metals chains, made to do manual labour and subjected to inhuman conditions. Seeing their plight, Shamshera signs a document by which the British would allow the Khamerans to escape if they were given double the quantity of gold that the upper castes had given them to stop the Khameran menace. Shamshera devises a plan to exit and in his pursuit of locating a secret tunnel, is wounded by bullets and caught by the British. Before leaving, he extracts a promise from his wife to disown him if he is caught. Shuddh Singh plays a trick and portrays Shamshera's attempt to escape as an act of cowardice "Bhagoda" and exhorts the Khamerans to pelt stones at herself and their unborn child and the entire tribe, endorses Shuddh Singh's version, resulting in Shamshera being killed by his own people. The only two persons who know the reality are Pir, Shamshera's trusted friend and his wife.

46. From here the story is fast forwarded to 25 years later when Balli, son of Shamshera, is a grown man and the character is portrayed as a mischievous man with a childlike heart, who trains children to steal small possessions of inhabitants of Kaza fort. He is loved and liked by the womenfolk and children in Kaza Fort but is condemned by the elders who taunt him as 'bhagode ki aulaad'. Balli is troubled by his legacy but dreams big of becoming a Police Officer. Pir trains him. Balli makes a fervent but unsuccessful attempt to impress Shuddh Singh but when he is beaten up, he confronts his mother to reveal his father's credentials. His mother and Pir narrate the actual story and the valour and goodness of Balli's father. Upon learning the reason and cause of his father's death, Balli makes an identical attempt to exit the fort through a tunnel and succeeds to reach the bank of the Azaad River. Basis the map given to him by Pir, Balli locates the Kaza town and lost companions of Shamshera, who ask Balli to return to earlier ways of looting gold in order to free the members of the tribe locked in the fort. This is a beginning of a new chapter for Balli. In the meantime, Sona a female dancer/entertainer who had met Balli earlier starts aiding him in his activities. She is captured by Shuddh Singh but saved by Balli/Shamshera, the new name and both get married and have a child. The upper caste Indians are troubled and angered by re-emergence of Shamshera and again requests Shuddh Singh and the new British commander, Freddie, to intervene. Balli accepts a challenge from Freddie for a sporting fight and defeats him. Balli tells Freddie to look into the background of Khamerans treaty. Despite heavy security at the event of Shuddh Singh's wedding, Balli and his friends loot the gold and jewellery. The group now lies low for a while, however, one of the band members betrays them and in the attack that follows most of the members of the gang are killed. Sona who had just delivered a child is also captured. As the movie proceeds, Balli steals the crown of the Queen in his zeal to exchange the same as a price for Sona's freedom. Before Freddie could help, in exchange of the crown, Shuddh Singh tricks and gets him, and the British soldiers are killed by the Indian police, who are under Shuddh Singh's control. During the fight between Shuddh Singh and Balli, members of Khameran tribe, who have assembled and locked behind the huge fort gates, learnt of the true story of valour and sacrifice of Shamshera from Pir and Balli's mother. This triggers them to break the shackles of their slavery and the fort gates and attack. Shuddh Singh is defeated by Balli and the entire tribe moves out of the Kaza fort.

47. The above is a short summary and narrative of the script and the film, respectively. Analysing the script of the Plaintiff and the film of the Defendants closely on a prima facie view, the Court finds that the KNCK script revolves around fights and battles between the Afghans who repeatedly made attempts to invade Indian territories and the Sikhs in the 18th century with the ultimate defeat of Ahmad Shah in 1763. The storyline portrays Sikhs as fearless independent people unwilling to compromise on the security of their motherland and willing to make any sacrifice for the sake of their people. The protagonist is the central character as in any other Bollywood film and in addition to the invasions, the theme centres around the lady love of the protagonist and the fight and the lust of the antagonist to achieve her. Kartar's father is not the central theme of the story albeit centre theme of the film Shamshera is a stray solitary incident in one of the scripts and does not flow with the story. Film, on the other hand, is a story of oppression of the lower caste at the hands of the upper caste and this thread runs in the entire film. From the beginning to the end, the oppression of the lower caste is inextricably woven into the story from their being outcasted into the desert and finally into the Jungle from the main town and the other elements include, being tricked and taken captive in the Kaza Fort; looting the upper caste in retaliation and finding a leader to fight the upper caste etc. The angle of a love relationship between the main character and the lady is not central to the movie. The entire focus is on achieving freedom from oppression, be it at the expense of looting gold and giving the same in exchange for freedom to the Indian policeman who is the central character in oppression and instigating the British. Character of the father Shamshera is central to the film and his past legacy is the backdrop. The film focusses on relationships between a father and a son spanning over a period of 25 years and a realisation by the son of the sacrifice and valour of his father whom he all through believed was a coward. The treatment meted out to the story of the film, in many ways, makes it different from the script
"""

summary = generate_summary(custom_case_text)
print("🧾 Custom Case Summary:\n", summary)
